
# 🗃️ Supermaket sales

In [0]:
%run ../_utils


## dependency

In [0]:
!pip install opendatasets

In [0]:
%run ../_create_database


## download dataset from kaggle

O dataset escolhido foi o [Supermarket sales](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales)

Como é um dataset estático, não faz sentido adicionar upsert e tampouco streaming. Mas conforme o padrão dos dados (adicional de horario de venda), na vida real poderia fazer sentido pelo menos um minibatch para ingestão dos dados

In [0]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales", data_dir="/FileStore/datasets/") # foi para file:/

In [0]:
%fs ls file:/FileStore/datasets/supermarket-sales


o arquivo foi salvo em `file:/FileStore/datasets/supermarket-sales/supermarket_sales - Sheet1.csv``


## Carregando o arquivo .csv para spark dataframe

aqui o .od fez eu usar o workspace files. [mais detalhes aqui](https://docs.databricks.com/en/files/index.html)

In [0]:
df = (
    spark.read.format("csv")
    .options(header="true", inferSchema=True)
    .load("file:/FileStore/datasets/supermarket-sales/supermarket_sales - Sheet1.csv")
)


## 🔎 First exploratory data analysis (descriptive)

In [0]:
display(df.take(10))

In [0]:
print(f"Total de linhas: {df.count()}")

In [0]:
df.printSchema()


O conjunto de dados é um dos registros históricos de vendas de uma empresa de supermercados, que foi registrado em 3 filiais diferentes ao longo de 3 meses. 

### schema
 - ``Invoice`` ID: Número de identificação da fatura de venda gerado pelo computador
 - ``Branch``: Filial do supermercado (3 filiais disponíveis identificadas por A, B e C).
 - ``City``: Localização dos supermercados
 - ``Customer`` type: Tipo de clientes, registrados como _Member_ para clientes que usam cartão de membro e Normal para aqueles sem cartão de membro.
 - ``Gender``: Tipo de gênero do cliente
 - ``Product`` line: Grupos de categorização geral de itens - Acessórios eletrônicos, Acessórios de moda, Alimentos e bebidas, Saúde e beleza, Casa e estilo de vida, Esportes e viagens
 - ``Unit price``: Preço de cada produto em USD ($)
 - ``Quantity``: Número de produtos comprados pelo cliente
 - ``Tax 5%``: Taxa de imposto de 5% para clientes que compraram
 - ``Total``: Preço total incluindo impostos
 - ``Date``: Data da compra (Registro disponível de janeiro de 2019 a março de 2019)
 - ``Time``: timestamp bugado, Hora da compra (das 10h às 21h)
 - ``Payment``: Método de pagamento usado pelo cliente na compra (3 métodos estão disponíveis - Dinheiro, Cartão de crédito e Ewallet)
 - ``cogs``: Custo dos bens vendidos
 - ``gross margin percentage``: Porcentagem de margem bruta
 - ``gross income``: Renda bruta
 - ``Rating``: Classificação de estratificação do cliente em sua experiência de compra geral (Em uma escala de 1 a 10)


## Saving data

salvando todos os .csv no dbf em arquivo parquet

Nossa staging poderia ser um S3 (aws) ou blob storage, mas vai ser diretamente no dbfs mesmo

In [0]:
file_name = "supermarket_sales"

In [0]:
save_dataframe(
    df,
    format_mode="parquet",
    target_location=f"dbfs:/FileStore/parquet/supermarket_sales/",
    table_name=file_name
)

In [0]:
%fs ls '/FileStore/parquet/supermarket_sales/'

In [0]:
dbutils.notebook.exit("OK")


#### test

In [0]:
display(spark.read.parquet("dbfs:/FileStore/parquet/supermarket_sales/").take(10))